In [85]:
from rdkit import Chem
from rdkit.Chem import QED
import pandas as pd
from tqdm import tqdm
import time
from rdkit.Chem import RDConfig
import os
import sys
sys.path.append(os.path.join(RDConfig.RDContribDir, 'SA_Score'))
# now you can import sascore!
import sascorer
import numpy as np
from rdkit.Chem import MACCSkeys

In [86]:
# fname = './eval_gtp2_20002_BTK_150.log'
fname = './eval_gtp2_5000_FGFR4_150.log'
f =  open(fname,'r')
lines = f.readlines()

predicts = []
truth = []
for line in lines:
    if 'startofsmiles' in line:
        # tmp = line.split('>')[1].strip()
        tmp = line.split('<|startofsmiles|>')
        # predicts.append(tmp[1].split(' ')[0].strip())
        predicts.append(tmp[-1].strip())

        # break
    elif 'Reference' in line:
        tmp = line.split('Reference smiles: ')[1].strip()
        truth.append(tmp)
    else:
        pass

nums = len(predicts)

In [87]:
import joblib
clf = joblib.load("SVC.m")

In [88]:
fname = f'../data/train/FGFR4.csv'
df_train = pd.read_csv(fname)
smiles_train_list = df_train['smiles'].values

In [89]:
positive = []
negtive = []
for i in range(nums):
    smile = predicts[i]
    if smile in smiles_train_list:
        continue
    try:
        mol=Chem.MolFromSmiles(smile)
    except Exception as e:
        continue

    if mol is None:
        continue

    fp = MACCSkeys.GenMACCSKeys(mol)
    fp_bits = fp.ToList()
    label = clf.predict(np.array(fp_bits).reshape(1,-1)).item()
    if label == 1:
        positive.append(smile)
    elif label == 0:
        negtive.append(smile)

[12:53:24] SMILES Parse Error: unclosed ring for input: 'Cc1ccccc1NC(=O)Cn1c(=O)n2c3c(sc2c2ccccc21)CN(C)CC2'
[12:53:24] SMILES Parse Error: extra close parentheses while parsing: Cc1ccccc1NC(=O)Cn1c(=O)n2c3c(sc2c2ccccc21)CN(C)CC2)c1ccccc1
[12:53:24] SMILES Parse Error: Failed parsing SMILES 'Cc1ccccc1NC(=O)Cn1c(=O)n2c3c(sc2c2ccccc21)CN(C)CC2)c1ccccc1' for input: 'Cc1ccccc1NC(=O)Cn1c(=O)n2c3c(sc2c2ccccc21)CN(C)CC2)c1ccccc1'
[12:53:24] Can't kekulize mol.  Unkekulized atoms: 15 16 18 19 20 21 22 23 24
[12:53:24] SMILES Parse Error: unclosed ring for input: 'Cc1ccccc1NC(=O)Cn1c(=O)n2c3c(sc2c2ccccc21)CN(C)CC2'
[12:53:24] SMILES Parse Error: extra open parentheses for input: 'CC(C)N1CCN(C(=O)c2ccc(CN(Cc3ccc4c(c3)C(=O)N(C3=CCCCC3)C4=O)cc2)CC1'
[12:53:25] Explicit valence for atom # 10 N, 4, is greater than permitted
[12:53:25] Explicit valence for atom # 10 N, 4, is greater than permitted
[12:53:25] SMILES Parse Error: unclosed ring for input: 'Cc1c=C(C)C(C)(C)c1ccc(S(=O)(=O)Oc2ccc(C(C)(C)C)

In [90]:
print(len(positive))
print(len(negtive))

624
600


In [91]:
output_positive = []
for smile in positive:
    output_positive.append([smile, 1])

output_negtive = []
for smile in negtive:
    output_negtive.append([smile, 0])

output_positive = pd.DataFrame(data=output_positive, columns=['smiles', 'label'])
output_positive.to_csv(f'./data/positive.csv', index=False)

output_negtive = pd.DataFrame(data=output_negtive, columns=['smiles', 'label'])
output_negtive.to_csv(f'./data/negtive.csv', index=False)
print('Saved.')

Saved.
